In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s6e1/sample_submission.csv
/kaggle/input/playground-series-s6e1/train.csv
/kaggle/input/playground-series-s6e1/test.csv


In [2]:
data_train = pd.read_csv("/kaggle/input/playground-series-s6e1/train.csv")
data_test = pd.read_csv("/kaggle/input/playground-series-s6e1/test.csv")

In [ ]:
data_train.head()


In [ ]:
data_train.info()

In [ ]:
data_train['age'].mean()

In [ ]:
data_train['age'].median()

In [ ]:
data_train = data_train.drop(columns = ['course', 'study_method', 'id'])

In [ ]:
data_train.isna().sum()


In [ ]:
data_train["gender"] = (data_train["gender"] == "male").astype(int)

data_train["internet_access"] = (data_train["internet_access"] != "no").astype(int)

data_train["sleep_quality"] = data_train["sleep_quality"].map({
    "poor": 0,
    "average": 1,
    "good": 2
})

data_train["facility_rating"] = data_train["facility_rating"].map({
    "low": 0,
    "medium": 1,
    "high": 2
})
data_train["exam_difficulty"] = data_train["exam_difficulty"].map({
    "easy": 0,
    "moderate": 1,
    "hard": 2
})


In [ ]:
data_train["exam_difficulty"].unique()


In [ ]:
data_train.head()

In [ ]:
results = data_train["exam_score"]
data_train = data_train.drop(columns = ["exam_score"])
results.head()

In [ ]:
feature_cols = data_train.columns.tolist()
print(feature_cols)

In [ ]:
##Scale the inputs
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(data_train)


In [ ]:
X_t = torch.tensor(X_train_scaled, dtype=torch.float32)
y_t = torch.tensor(results.values, dtype=torch.float32).view(-1, 1)

In [ ]:
##Training the model

model = nn.Sequential(
    nn.Linear(9, 64),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Dropout(0.2),

    nn.Linear(32, 1)   
)

In [ ]:
loss_f = nn.MSELoss()
EPOCHS = 1000
ALPHA = 0.001
optimizer = optim.Adam(model.parameters(), lr= ALPHA)
#A list to store losses
losses = []

In [ ]:
for epoch in range(EPOCHS):
    #Reset the gradient
    optimizer.zero_grad()
    #Make the predictions
    outputs = model(X_t)
    #Calculate the loss
    loss = loss_f(outputs, y_t)
    losses.append(loss.item())
    #Calculate adjustments (backward pass)
    loss.backward()
    #Update the model's parameters
    optimizer.step()
    if (epoch + 1) % 50 == 0:
        print(f"The loss is {loss} in the epoch {epoch + 1}")

In [ ]:
plt.plot(range( EPOCHS), losses)
plt.xlabel = "Epochs"
plt.ylabel = "Loss"
plt.title = "Learning Curve"
plt.show()

In [ ]:
data_test = data_test[feature_cols]
data_test["gender"] = (data_test["gender"] == "male").astype(int)

data_test["internet_access"] = (data_test["internet_access"] != "no").astype(int)

data_test["sleep_quality"] = data_test["sleep_quality"].map({
    "poor": 0,
    "average": 1,
    "good": 2
})

data_test["facility_rating"] = data_test["facility_rating"].map({
    "low": 0,
    "medium": 1,
    "high": 2
})
data_test["exam_difficulty"] = data_test["exam_difficulty"].map({
    "easy": 0,
    "moderate": 1,
    "hard": 2
})


X_test_scaled = scaler.transform(data_test)
X_test_t = torch.tensor(X_test_scaled, dtype=torch.float32)



In [ ]:
model.eval()
with torch.no_grad():
    predictions = model(X_test_t)


In [ ]:
final_preds = predictions.cpu().numpy().flatten()

# 2. Boyut kontrolü (İsteğe bağlı ama hayat kurtarır)
print(f"Tahmin sayısı: {len(final_preds)}")
print(f"Sample satır sayısı: {len(sample)}")

# 3. Atama yap
data_test["exam_score"] = final_preds

# 4. Kaydet (Yorum satırını açmayı unutma!)
data_test.to_csv("submission.csv", index=False)

print("Dosya kaydedildi! İlk 5 tahmin:")
print(data_test.head())

In [ ]:
len(data_test) == len(predictions)

In [ ]:
data_test["exam_score"] = predictions
data_test.to_csv("submission.csv", index=False)

